# RAG - Implementation and Evaluation

## Goal:

- Build notebook to do RAG wth LangChain
- Evaluation

By: Hasan Rafiq

# Installing the package

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user
!pip install langchain langchain-core langchain-google-vertexai faiss-cpu ragas

# Authenticate again

In [3]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [4]:
PROJECT_ID = "<YOUR PROJECT ID>"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [110]:
from google.cloud import aiplatform
# from vertexai.preview.language_models import TextGenerationModel, ChatModel
from vertexai.generative_models import GenerativeModel, Part, ChatSession, Image, FunctionDeclaration, Tool, Content
from langchain_google_vertexai import VertexAI
import http.client
import typing
import urllib.request
import pandas as pd

from google.cloud import aiplatform, storage
from vertexai.preview.language_models import TextGenerationModel, ChatModel, TextEmbeddingModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pprint
import re
import io

from langchain.llms import VertexAI
from langchain import hub
from langchain.text_splitter import TextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, GCSFileLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness
)
from datasets import Dataset
from ragas import evaluate

aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Implement a RAG

## Source text

In [6]:
document = ["""
Information retrieval using AI and LLMs
Vertex AI Search brings together the power of deep information retrieval, state-of-the-art natural language processing, and the latest in large language model (LLM) processing to understand user intent and return the most relevant results for the user.

With Vertex AI Search, you can build a Google-quality search app on your own data and embed a search bar in your web pages or app.

With Recommendations, you can build a recommendations app on your own data that will suggest content similar to the content that the user is currently viewing.

Note: The generic recommendations feature is a Preview offering, subject to the "Pre-GA Offerings Terms" of the GCP Service Specific Terms. Pre-GA products and features may have limited support, and changes to pre-GA products and features may not be compatible with other pre-GA versions. For more information, see the launch stage descriptions. Further, by using this feature, you agree to the Generative AI Preview terms and conditions ("Preview Terms"). For this feature, you can process personal data as outlined in the Cloud Data Processing Addendum, subject to applicable restrictions and obligations in the Agreement (as defined in the Preview Terms).
An easy experience to get started
Vertex AI Search makes it easy to get started with high-quality search or recommendations based on data that you provide. As part of the setup experience, you can:

Use your existing Google Account or sign up for one.
Use your existing Google Cloud project or create one.
Create an app and attach a data store to it. Provide data to search or recommend by entering the URLs for your website content, importing your data from BigQuery or Cloud Storage, or importing FHIR R4 data from Cloud Healthcare API, or uploading through RESTful CRUD APIs. Syncing data from Jira, Salesforce, or Confluence is available in Preview with allowlist.
Embed JavaScript widgets and API samples to integrate search or recommendations into your website or applications.
Data stores and apps
With Vertex AI Search, you create a search or recommendations app and attach it to a data store. You import your data into a data store and index your data. Apps and data stores have a one-to-one relationship.

There are various kinds of data stores that you can create, based on the type of data you use. Each data store can contain one type of data:

Website data: You can provide domains such as yourexamplewebsite.com/faq and yourexamplewebsite.com/events and enable search or recommendations over the content at those domains.
Structured data: A data store with structured data enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files. For example, you can enable search or recommendations over a product catalog for your ecommerce experience, a movie catalog for movie search or recommendations, or a directory of doctors for provider search or recommendations.
Unstructured data: An unstructured data store enables semantic search or recommendations over data such as documents and images. For example, a financial institution can enable search or recommendations over their private corpus of financial research publications, or a biotech company can enable search or recommendations over their private repository of medical research.
Healthcare data: A healthcare data store enables semantic search over healthcare FHIR R4 data imported from Cloud Healthcare API. For example, a healthcare provider can search over a patient's clinical history using exploratory queries.
"""]

## Create helper functions

In [64]:
# Initialize Gemini model
model = VertexAI( max_output_tokens=2048, model="gemini-1.0-pro", top_p = 1, temperature = 0)
llm_embeddings = VertexAIEmbeddings()

def chunk_text(input_texts, separator="", chunk_size=100, overlap=20):
  ## Create split of texts on a chunk
  text_splitter = CharacterTextSplitter(separator="", chunk_size=chunk_size, chunk_overlap=overlap)
  chunks = text_splitter.create_documents(input_texts)

  return chunks

def create_vector_store(chunks):
  ## Create vector store from embeddings
  db = FAISS.from_documents(chunks, llm_embeddings)

  return db

# def query_vector_store(query, db, top_n_docs=5):
#   docs = db.similarity_search(query, k=top_n_docs)

#   return docs

def query_vector_store(query, db, top_n_docs, multiquery):
  if multiquery:
    retriever_from_llm = MultiQueryRetriever.from_llm(
      retriever=db.as_retriever(search_kwargs={"k": top_n_docs}), llm=model
    )
    docs = retriever_from_llm.get_relevant_documents(query=query)
  else:
    retriever = db.as_retriever(search_kwargs={"k": top_n_docs})
    docs = retriever.get_relevant_documents(query)

  return docs

def create_with_refine(input_chunks, title):
  prompt_template = """Respond as per following instructions:
  {text}
  CONCISE SUMMARY:"""
  prompt = PromptTemplate.from_template(prompt_template)

  refine_template = (
      """
      You are a QnA bot meant to answer questions ONLY from the contexts provided below and NO PRIOR knowledge

      Question:
      -----------
      {query}
      -----------

      We have provided a response version of the response up to a certain point: {existing_answer}
      You have the opportunity to refine the existing response response with below contexts
      (only if needed) with some more context as specified below.

      Rules:
      ------------
      Given the new context and the query, refine the response further as necessary for answering the query but
      never include the context as is in the response, you should ONLY and ONLY use the knowledge provided in contexts.
      DONT add any information which is not present in contexts.
      Make sure that the content is always relevant to the original query.
      If the context provided isn't useful, dont make any change to the response, just return the current response.
      ------------

      Context:
      ------------
      {text}
      ------------
      """
  ).replace('{query}', title)

  refine_prompt = PromptTemplate.from_template(refine_template)
  chain = load_summarize_chain(
      llm=model,
      chain_type="refine",
      question_prompt=prompt,
      refine_prompt=refine_prompt,
      return_intermediate_steps=True,
      input_key="input_documents",
      output_key="output_text"
  )
  result = chain({"input_documents": input_chunks}, return_only_outputs=True)

  context = []
  for ic in input_chunks:
    context.append( ic.page_content )

  return result['intermediate_steps'], result['output_text'], context

def run_query(query, db, top_n_docs=5, multiquery=False):
  docs = query_vector_store(query, db, top_n_docs, multiquery)

  ## Create article using top N docs
  intermediate_steps, final_text, contexts = create_with_refine(docs, query)

  return intermediate_steps, final_text, contexts

In [97]:
top_n_docs = 6 # @param {type:"integer"} ## Number of similar chunks used to create content

## Create chunks from array of texts
separator = " "
chunks = chunk_text(document, separator, chunk_size=250, overlap=20)
len(chunks)

16

In [98]:
## Create vector store from chunks
db = create_vector_store(chunks)

## Test RAG

In [115]:
model_answers = []
retrieved_contexts = []

In [116]:
query = "What type of data can data stores contain in Vertex AI Search ? Give an elaborated answer" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

' Vertex AI Search can store various types of data, including:\n\n- **Text:** This includes unstructured text data such as documents, articles, emails, and web pages. Vertex AI Search uses natural language processing (NLP) to understand the meaning of text data and extract relevant information.\n\n\n- **Structured data:** This includes data that is organized in a tabular format, such as spreadsheets, databases, and CSV files. Vertex AI Search can index structured data and make it searchable by specific fields or columns. This enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files.\n\n\n- **Images:** Vertex AI Search can index and search images based on their visual content. It uses computer vision technology to extract features from images and match them to relevant queries.\n\n\n- **Audio:** Vertex AI Search can index and search audio files, such as podcasts, music, and voice recordings. It uses speech recognition technology to transcr

In [117]:
query = "What does Vertex search support in terms of Healthcare data ?" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

" **Healthcare Data:**\n- Semantic search over healthcare FHIR R4 data from Cloud Healthcare API.\n- Suitable for healthcare organizations, research institutions, or biotech companies.\n- For example, a healthcare provider can search over a patient's clinical history using exploratory queries.\n- Vertex AI Search can also be used to search over other types of healthcare data, such as medical images, genomic data, and electronic health records.\n- It leverages the power of deep information retrieval, state-of-the-art natural language processing, and the latest in large language model (LLM) processing to understand user intent and retrieve relevant information from healthcare data sources.\n- Vertex AI Search supports structured data, which enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files.\n- You can import FHIR R4 data from Cloud Healthcare API to enable semantic search over healthcare data."

In [118]:
query = "Which all services are currently in preview in Vertex Search ?" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

' Vertex AI Search is currently in preview and offers the following services:\n\n- Search: Build a Google-quality search app on your own data and embed a search bar in your web pages or app.\n- Recommendations: Build a recommendations user intent and return the most relevant results for the user.\n- Data sources: Create search indexes from various data sources, including Salesforce, Jira, Confluence, healthcare FHIR R4 data imported from Cloud Healthcare API, and private repositories of medical research.'

In [119]:
pprint.pp(model_answers)

[' Vertex AI Search can store various types of data, including:\n'
 '\n'
 '- **Text:** This includes unstructured text data such as documents, '
 'articles, emails, and web pages. Vertex AI Search uses natural language '
 'processing (NLP) to understand the meaning of text data and extract relevant '
 'information.\n'
 '\n'
 '\n'
 '- **Structured data:** This includes data that is organized in a tabular '
 'format, such as spreadsheets, databases, and CSV files. Vertex AI Search can '
 'index structured data and make it searchable by specific fields or columns. '
 'This enables semantic search or recommendations over structured data such as '
 'a BigQuery table or NDJSON files.\n'
 '\n'
 '\n'
 '- **Images:** Vertex AI Search can index and search images based on their '
 'visual content. It uses computer vision technology to extract features from '
 'images and match them to relevant queries.\n'
 '\n'
 '\n'
 '- **Audio:** Vertex AI Search can index and search audio files, such as '
 'po

# Evaluate RAG with RAGAs

## Generate a test set

In [120]:
import google.auth

from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from ragas.llms.base import LangchainLLMWrapper

config = {
    "project_id": "hasanrafiq-test-331814",
}

# authenticate to GCP
creds, _ = google.auth.default(quota_project_id=config['project_id'])

llm = ChatVertexAI(top_k=40, top_p=1, temperature=0, credentials=creds)
ragas_vertexai_llm = wrapper = LangchainLLMWrapper(llm)
vertexai_embeddings = VertexAIEmbeddings(credentials=creds)

In [121]:
questions = [
    "What type of data can data stores contain in Vertex AI Search ?",
    "What does Vertex search support in terms of Healthcare data ?",
    "Which all services are currently in preview in Vertex Search ?"
]

In [122]:
correct_answers = [
    ["""Website data, structured data, unstructured data, health care data"""],
    ["""You can do a semantic search over healthcare FHIR R4 data imported from Cloud Healthcare API. For example, a healthcare provider can search over a patient's clinical history using exploratory queries."""],
    ["""Currently Syncing data from Jira, Salesforce, or Confluence is available in Preview with allowlist and generic recommendation feature is also in preview"""]
]

In [123]:
test_set = {
 'question': questions,
 'ground_truths': correct_answers,
 'answer': model_answers,
 'contexts': retrieved_contexts
}

metrics = [
        # answer_correctness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall
]

## Switch to VertexAI models
for m in metrics:
    # change LLM for metric
    m.__setattr__("llm", ragas_vertexai_llm)

    # check if this metric needs embeddings
    if hasattr(m, "embeddings"):
        # if so change with VertexAI Embeddings
        m.__setattr__("embeddings", vertexai_embeddings)

In [124]:
df_eval = Dataset.from_dict(test_set)

score = evaluate(df_eval, metrics=metrics)
score.to_pandas()

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

,question,ground_truths,answer,contexts,ground_truth,answer_relevancy,context_precision,context_recall,context_entity_recall
0,What type of data can data stores contain in V...,"[Website data, structured data, unstructured d...",Vertex AI Search can store various types of d...,[Information retrieval using AI and LLMs\nVert...,"Website data, structured data, unstructured da...",0.947733,0.325000,1.0,0.5
1,What does Vertex search support in terms of He...,[You can do a semantic search over healthcare ...,**Healthcare Data:**\n- Semantic search over ...,"[ublications, or a biotech company can enable ...",You can do a semantic search over healthcare F...,0.000000,0.876667,1.0,0.5
2,Which all services are currently in preview in...,"[Currently Syncing data from Jira, Salesforce,...",Vertex AI Search is currently in preview and ...,"[rom Jira, Salesforce, or Confluence is availa...","Currently Syncing data from Jira, Salesforce, ...",0.814847,1.000000,1.0,1.0
